In [ ]:
# Author: Emma Ding (2024)
# Model Development File
# Kaggle Environment
# GPU T4 x2 Accelerator

import os
from IPython.display import FileLink
from PIL import Image

import numpy as np 
import pandas as pd

import keras
import tensorflow as tf
import torch
from tensorflow.random import gamma as tf_random_gamma

!pip install ultralytics
from ultralytics import YOLO

In [ ]:
# Create directory for processed ASL MNIST Dataset
os.makedirs("/kaggle/working/sign-language-data")

# Create training and validation directories
os.makedirs("/kaggle/working/sign-language-data/train")
os.makedirs("/kaggle/working/sign-language-data/val")

# Create one folder for each class
for i in range(26):
    os.makedirs("/kaggle/working/sign-language-data/train/{label}".format(label=i))
    os.makedirs("/kaggle/working/sign-language-data/val/{label}".format(label=i))

# Remove J and Z classes
for i in (9,25):
    os.removedirs("/kaggle/working/sign-language-data/train/{label}".format(label=i))
    os.removedirs("/kaggle/working/sign-language-data/val/{label}".format(label=i))

In [ ]:
# Import the ASL MNIST Datasets
# MNIST_train is 80, MNIST_test is 20
sl_df_train=pd.read_csv('/kaggle/input/sign-language-mnist/sign_mnist_train/sign_mnist_train.csv')
sl_df_val=pd.read_csv('/kaggle/input/sign-language-mnist/sign_mnist_test/sign_mnist_test.csv')

In [ ]:
# Split the dataset into validation and test, 10:10
sl_df_val=sl_df_val[:len(sl_df_val.index)//2]
sl_df_test=sl_df_val[len(sl_df_val.index)//2:]

In [ ]:
# Transforming the pixel-level data from MNIST to images
sl_df_train.apply(lambda x: Image.fromarray(np.reshape(np.uint8(x.iloc[1:]), (28,28)),'L').save("/kaggle/working/sign-language-data/train/{label}/{name}.png".format(label=x.label,name=x.name)),axis=1)
sl_df_val.apply(lambda x: Image.fromarray(np.reshape(np.uint8(x.iloc[1:]), (28,28)),'L').save("/kaggle/working/sign-language-data/val/{label}/{name}.png".format(label=x.label,name=x.name)),axis=1)

In [ ]:
# Load pretrained classification YOLO v8 model
model = YOLO("yolov8n-cls.pt")

# Fine tune the YOLO v8 model on the training and validation data for 5 epochs
results = model.train(data="/kaggle/working/sign-language-data", epochs=5, imgsz=784,save_dir="/kaggle/working/")

In [ ]:
# Save training results
!zip -r run.zip /kaggle/working/runs
FileLink(r'run.zip')